<a href="https://colab.research.google.com/github/Janina712/PUS2024_JBoecher/blob/main/HW4/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. imports**

In [1]:
!wget https://raw.githubusercontent.com/fedhere/PUS2024/refs/heads/main/HW3/evalChisq.py

--2024-10-01 15:46:04--  https://raw.githubusercontent.com/fedhere/PUS2024/refs/heads/main/HW3/evalChisq.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 325 [text/plain]
Saving to: ‘evalChisq.py.1’

evalChisq.py.1      100%[===================>]     325  --.-KB/s    in 0s      

2024-10-01 15:46:05 (5.30 MB/s) - ‘evalChisq.py.1’ saved [325/325]



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import requests
from io import BytesIO
from scipy.stats import chisquare
import calendar
from evalChisq import evalChisq

**1. load data**

In [3]:
datestring = '2019'

# Define the URL of the zip file
url = "https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip"

# Download the zip file
response = requests.get(url)
zip_file = zipfile.ZipFile(BytesIO(response.content))

In [4]:
zip_file.extractall() #extract all the files in the zipped folder locally

In [5]:
!ls

2019-citibike-tripdata	evalChisq.py  evalChisq.py.1  __MACOSX	__pycache__  sample_data


In [6]:
!ls 2019-citibike-tripdata

10_October   12_December  2_February  4_April  6_June  8_August
11_November  1_January	  3_March     5_May    7_July  9_September


In [7]:
!ls 2019-citibike-tripdata/5_May

201905-citibike-tripdata_1.csv	201905-citibike-tripdata_2.csv


In [8]:
# Read the CSV file from the zip file
cbMay19 = pd.read_csv(datestring + '-citibike-tripdata/5_May/' + datestring + '05-citibike-tripdata_1.csv')

In [9]:
cbMay19.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1


# **Idea**

I'm interested to know if people who use citibikes on a regular basis (e.g., for commuting to work) are more likely to be annual subscribers than people who use citibikes casually on the weekend. Maybe my goal is to see if different pricing models are more attractive for different types of riders (casual vs. commuter) to then customize my advertisement for different audiences.

# **Hypothesis**

**H_0:** Riders who use citibikes for commuting (trips starting Monday - Friday between 6:30-9:30 am and 3:30-6:30 pm) have an annual membership (subscribers) as often as leisure riders (trips starting on Saturday/Sundays).


**H_A:** Riders who use citibikes for commuting (trips starting Monday - Friday between 6:30-9:30 am and 3:30-6:30 pm) have an annual membership (subscribers) more or less often than leisure riders (trips starting on Saturday/Sundays).


# **Procedure**

- identify commuting: starttime column: import a calendar to find out which dates were weekdays; subset data to include only rides that started on weekdays between 6:30 and 9:30 am and between 3:30 and 6:30 pm

- identify leisure: starttime column: import a calendar to find out which dates were weekends; subset data to include only rides that started on weekends

- get subscriber vs. customer: usertype column


**Identify Business Day vs. Weekend Day**

In [10]:
day_col = []
year = 2019
month = 5

for i in range(0,len(cbMay19)):

# get day of the week
  day = int(cbMay19['starttime'][i][8:10])
  day_of_week = calendar.weekday(year,month,day)

  # assign business or weekend day to list
  if day_of_week < 5:
    day_col.append('Business Day')
  else:
    day_col.append('Weekend Day')

# add to df
cbMay19.insert(3, 'Weekday', day_col)

**Identify Rush Hour vs. Off Peak**

In [11]:
time_col = []

for i in range(0,len(cbMay19)):

  # get time of the day
  hour = int(cbMay19['starttime'][i][11:13])
  minute = int(cbMay19['starttime'][i][14:16])

  # assign rush hour or off peak to list
  if ((hour >=6) & (hour <10)) | ((hour >=3) & (hour <7)):
    if (hour == 6) & (minute >= 30):
      time_col.append('Rush Hour')
    elif (hour == 9) & (minute < 30):
      time_col.append('Rush Hour')
    elif (hour == 3) & (minute >= 30):
      time_col.append('Rush Hour')
    elif (hour == 6) & (minute < 30):
      time_col.append('Rush Hour')
    else:
      time_col.append('Rush Hour')
  else:
    time_col.append('Off Peak')

# add to df
cbMay19.insert(4, 'Time of Day', time_col)

**Define Commuter vs. Leisure Rider**

In [12]:
rider_col = []

for i in range(0,len(cbMay19)):

  # assign commuter vs. leisure rider to list
  if ((cbMay19['Weekday'][i] ==  'Business Day') & (cbMay19['Time of Day'][i] ==  'Rush Hour')):
    rider_col.append('Commuter')
  else:
    rider_col.append('Leisure Rider')

# add to df
cbMay19.insert(5, 'Rider', rider_col)

In [13]:
cbMay19.head()

,tripduration,starttime,stoptime,Weekday,Time of Day,Rider,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,Business Day,Off Peak,Leisure Rider,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,Business Day,Off Peak,Leisure Rider,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,Business Day,Off Peak,Leisure Rider,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,Business Day,Off Peak,Leisure Rider,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,Business Day,Off Peak,Leisure Rider,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1


# **Statistical Test**

I have two groups (commuter vs. leisure rider) and a categorical outcome variable (subscriber or customer).

Test: Chi-Square Test

Function: scipy.stats.chisquare()

Reject H_0 if *p* < .05.

**Build Contingency Table**

In [14]:
commuters = cbMay19[cbMay19["Rider"] == "Commuter"]
leisure_riders = cbMay19[cbMay19["Rider"] == "Leisure Rider"]

In [15]:
print(commuters['usertype'].value_counts()['Subscriber'])
print(commuters['usertype'].value_counts()['Customer'])
print(len(commuters))

214696
8354
223050


In [16]:
print(leisure_riders['usertype'].value_counts()['Subscriber'])
print(leisure_riders['usertype'].value_counts()['Customer'])
print(len(leisure_riders))

664548
112402
776950


**Contingency Table (count)**

|Rider |     Subscriber          | Customer     |Total|                   
|---------------------------|:---------------------:|------------------|---------------------------|
| Commuter               | $214696$           | $8354$      | $223050$                       |
| Leisure Rider            | $664548$           | $112402$     | $776950$                       |
| Total                     | $879244$               |  $120756$         | $1000000$                       |

**As percentage**

In [17]:
commuter = [214696/(223050/100), 8354/(223050/100)]
leisure = [664548/(776950/100), 112402/(776950/100)]
total = [879244/(1000000/100), 120756/(1000000/100)]

**Contingency Table (percentage)**

|Rider |     Subscriber          | Customer     |Total|                   
|---------------------------|:---------------------:|------------------|---------------------------|
| Commuter               | $96.3$           | $3.7$      | $100$                       |
| Leisure Rider            | $85.5$           | $14.5$     | $100$                       |
| Total                     | $87.9$               |  $12.1$         | $100$                       |

**Chi-Square Test**

In [18]:
Ntot = 100000
sample_values = np.array([[214696, 8354], [664548, 112402]])
print (evalChisq(sample_values))

18762.852481440717


For this test, the degrees of freedom is 1 (e.g., 2 test groups).

The chi square statistics is 18763, which is larger than 3.84 (= critical value at 0.05 significance). This means that the chi square statistics is clearly within the region where the null hypothesis can be rejected. Thus, we reject the null hypothesis.  

**Conclusion:** Riders who use citibikes for commuting (trips starting Monday - Friday between 6:30-9:30 am and 3:30-6:30 pm) have an annual membership (subscribers) at a different rate than leisure riders (trips starting on Saturday/Sundays). Based on the contingency table, they have annual memberships more often compared to leisure riders.